In [ ]:
import multiprocessing 

import six
import numpy as np
import tensorflow.compat.v2 as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

!pip install git+https://github.com/google/qkeras

In [ ]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

from qkeras import *

In [ ]:
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

def get_one_hot(targets, nb_classes):
    res = np.eye(nb_classes)[np.array(targets).reshape(-1)]
    return res.reshape(list(targets.shape)+[nb_classes])
#train_labels, test_labels = get_one_hot(train_labels, 10), get_one_hot(test_labels, 10)#

train_images = train_images.reshape(train_images.shape + (1,)).astype("float32")
test_images = test_images.reshape(test_images.shape + (1,)).astype("float32")

In [ ]:
from qkeras import *
import qkeras
from qkeras.utils import *

def CreateQModel(shape, nb_classes, intBits):
    x = x_in = Input(shape)

    x = QConv2D(32, (3, 3),
                kernel_quantizer="quantized_bits(17, {} , alpha=1)".format(intBits),
                bias_quantizer="quantized_bits(17, {} , alpha = 1)".format(intBits),
                name='c1')(x)
    x = QActivation('quantized_relu(17, {})'.format(intBits))(x)
    x = MaxPooling2D((2,2))(x)

    x = QConv2D(64, (3, 3),
                kernel_quantizer="quantized_bits(17, {} , alpha=1)".format(intBits),
                bias_quantizer="quantized_bits(17, {} , alpha = 1)".format(intBits),
                name="c2")(x)
    x = QActivation('quantized_relu(17, {})'.format(intBits))(x)
    x = MaxPooling2D((2,2))(x)

    x = QConv2D(64, (3, 3),
                kernel_quantizer="quantized_bits(17, {} , alpha=1)".format(intBits),
                bias_quantizer="quantized_bits(17, {} , alpha = 1)".format(intBits),
                name='c3')(x)
    x = QActivation('quantized_relu(17, {})'.format(intBits))(x)

    x = Flatten(name="flatten")(x)

    x = QDense(64,
        kernel_quantizer="quantized_bits(17, {} , alpha=1)".format(intBits),
        bias_quantizer="quantized_bits(17, {} , alpha=1)".format(intBits),
        name="dense3")(x)
    x = QActivation('quantized_relu(17, {})'.format(intBits))(x)

    x = QDense(nb_classes,
        kernel_quantizer="quantized_bits(17, {} , alpha=1)".format(intBits),
        bias_quantizer="quantized_bits(17, {} , alpha=1)".format(intBits),
        name="dense4")(x)
    x = Activation("softmax", name="softmax")(x)


    model = Model(inputs=x_in, outputs=x)
    
    return model

from tensorflow.keras.optimizers import Adam
model = CreateQModel(x_train.shape[1:], y_train.shape[-1], 1)
model.compile(
    loss="categorical_crossentropy",
    #loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=Adam(0.1),
    #optimizer='sgd',
    metrics=["accuracy"])
history = model.fit(x_train, y_train, epochs=10, batch_size=256, validation_data=(x_test[:5000], y_test[:5000]), verbose=True) # callbacks=[callback])
model_save_quantized_weights(model)
print ("Done")



# Tensorflow Colab file with some modifications

In [ ]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

from qkeras import *

In [ ]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

In [ ]:
intBits = 1
precision = 32

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
#model.add(QConv2D(32, (3,3), kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
#        bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
#        activation='quantized_relu({}, {})'.format(precision, intBits),
#        name='c1'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
#model.add(QConv2D(64, (3,3), kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
#        bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
#        activation='quantized_relu({}, {})'.format(precision, intBits),
#        name='c2'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
#model.add(QConv2D(64, (3,3), kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
#        bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
#        activation='quantized_relu({}, {})'.format(precision, intBits),
#        name='c3'))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
#model.add(QDense(64, kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
#        bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
#        name='d1'))
#model.add(QActivation('quantized_relu({}, {})'.format(precision, intBits)))
model.add(layers.Activation('relu'))
model.add(layers.Dense(10))

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images[:48000], train_labels[:48000], epochs=15, 
                    validation_data=(train_images[48000:], train_labels[48000:]))

In [ ]:
import qkeras
from qkeras import *
from qkeras.utils import *

In [ ]:
model_save_quantized_weights(model)
print ("Done")

In [ ]:
model.evaluate(test_images, test_labels)

In [ ]:
### Evaluation in a loop
intBits = 1
histories = []
mymodels = []
for precision in [25]:
  model = models.Sequential()
  #model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
  model.add(QConv2D(32, (3,3), kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        activation='quantized_relu({}, {})'.format(precision, intBits),
        name='c1'))
  model.add(layers.MaxPooling2D((2, 2)))
  #model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(QConv2D(64, (3,3), kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        activation='quantized_relu({}, {})'.format(precision, intBits),
        name='c2'))
  model.add(layers.MaxPooling2D((2, 2)))
  #model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(QConv2D(64, (3,3), kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        activation='quantized_relu({}, {})'.format(precision, intBits),
        name='c3'))

  model.add(layers.Flatten())
  #model.add(layers.Dense(64, activation='relu'))
  model.add(QDense(64, kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        name='d1'))
  model.add(QActivation('quantized_relu({}, {})'.format(precision, intBits)))
  model.add(QDense(10, kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        name='d2'))

  model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

  history = model.fit(train_images[:48000], train_labels[:48000], epochs= 3 + int((precision/4) - 2), 
                    validation_data=(train_images[48000:], train_labels[48000:]), verbose=False)
  
  model_save_quantized_weights(model)
  print ("Done")

  model.evaluate(test_images, test_labels)

  mymodels.append(model)
  histories.append(history)

In [ ]:
histories[4].history['val_accuracy']

# Only for 24 bit

In [ ]:
### Evaluation in a loop
precision = 25
intBits = 1
histories = []
mymodels = []
for epoch in range(3, 15):
  model = models.Sequential()
  #model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
  model.add(QConv2D(32, (3,3), kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        activation='quantized_relu({}, {})'.format(precision, intBits),
        name='c1'))
  model.add(layers.MaxPooling2D((2, 2)))
  #model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(QConv2D(64, (3,3), kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        activation='quantized_relu({}, {})'.format(precision, intBits),
        name='c2'))
  model.add(layers.MaxPooling2D((2, 2)))
  #model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(QConv2D(64, (3,3), kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        activation='quantized_relu({}, {})'.format(precision, intBits),
        name='c3'))

  model.add(layers.Flatten())
  #model.add(layers.Dense(64, activation='relu'))
  model.add(QDense(64, kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        name='d1'))
  model.add(QActivation('quantized_relu({}, {})'.format(precision, intBits)))
  model.add(QDense(10, kernel_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        bias_quantizer="quantized_bits({}, {} , alpha=1)".format(precision, intBits),
        name='d2'))

  model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

  history = model.fit(train_images[:48000], train_labels[:48000], epochs= epoch, 
                    validation_data=(train_images[48000:], train_labels[48000:]), verbose=False)
  
  model_save_quantized_weights(model)
  print ("Done")

  model.evaluate(test_images, test_labels)

  mymodels.append(model)
  histories.append(history)